## Libraries and Setup

In [1]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI, OpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.document_loaders import CSVLoader
from langchain.chains import RetrievalQA
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator

from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv())

In [3]:
llm_model = "gpt-3.5-turbo"

In [4]:
embeddings = OpenAIEmbeddings()

In [5]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')

In [6]:
loader

# Vector Store Index Creator

In [9]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

In [10]:
query ="Please create a markdown table listing all the shirts that offer sun protection. \
For each shirt, include the name, sun protection rating, and any other relevant details. After the table, provide a brief \
summary of the features of each shirt."

In [11]:
llm_replacement_model = OpenAI(
    temperature=0, 
    model='gpt-3.5-turbo-instruct'
)

response = index.query(
    query, 
    llm = llm_replacement_model
)

In [12]:
display(Markdown(response))



| Name | Sun Protection Rating | Additional Details |
| --- | --- | --- |
| Sun Shield Shirt by | UPF 50+ | Made of 78% nylon and 22% Lycra Xtra Life fiber, wicks moisture, fits comfortably over swimsuit, abrasion resistant |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ | Made of 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ | Made of 52% polyester and 48% nylon, SunSmart technology blocks 98% of UV rays, wrinkle-free, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ | Made of 71% nylon and 29% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |

The Sun Shield Shirt by offers UPF 50+ sun protection and is made of a high-performance fabric that wicks moisture and is abrasion resistant. The Men's Tropical Plaid Short-Sleeve Shirt is

# Step by Step

In [13]:
loader = CSVLoader(file_path=file, encoding='utf-8')

In [14]:
docs = loader.load()
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.")

In [15]:
embed = embeddings.embed_query("Hi my name is Saad Tariq")

In [16]:
print(len(embed))

1536


In [17]:
print(embed[:5])

[-0.010844825766980648, -0.012418035417795181, -0.02741510234773159, -0.03672540560364723, -0.009078189730644226]


## Vector Store

In [18]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [19]:
query = "Please suggest a shirt with sunblocking"

In [20]:
docs = db.similarity_search(query)

In [21]:
len(docs)

4

In [22]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \r\n\r\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\r\n\r\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\r\n\r\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\r\n\r\nSun Protection That Won\'t Wear Off\r\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

## Retriever

In [23]:
retriever = db.as_retriever()

In [24]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [25]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [26]:
response = llm.invoke(f"{qdocs} Question: Please create a markdown table listing all the shirts that offer sun protection. \
For each shirt, include the name, sun protection rating, and any other relevant details. After the table, provide a brief \
summary of the features of each shirt.") 
response

AIMessage(content="| Shirt Name                        | Sun Protection Rating | Additional Details                                                                                           |\n|-----------------------------------|-----------------------|--------------------------------------------------------------------------------------------------------------|\n| Sun Shield Shirt                  | UPF 50+              | Wicks moisture, fits comfortably over swimsuit, abrasion resistant                                           |\n| Men's Plaid Tropic Shirt          | UPF 50+              | Wrinkle-free, evaporates perspiration, front and back cape venting, two front bellows pockets                |\n| Men's TropicVibe Shirt            | UPF 50+              | Wrinkle resistant, front and back cape venting, two front bellows pockets                                    |\n| Men's Tropical Plaid Short-Sleeve | UPF 50+              | Wrinkle-resistant, front and back cape venting, two f

In [27]:
display(Markdown(response.content))

| Shirt Name                        | Sun Protection Rating | Additional Details                                                                                           |
|-----------------------------------|-----------------------|--------------------------------------------------------------------------------------------------------------|
| Sun Shield Shirt                  | UPF 50+              | Wicks moisture, fits comfortably over swimsuit, abrasion resistant                                           |
| Men's Plaid Tropic Shirt          | UPF 50+              | Wrinkle-free, evaporates perspiration, front and back cape venting, two front bellows pockets                |
| Men's TropicVibe Shirt            | UPF 50+              | Wrinkle resistant, front and back cape venting, two front bellows pockets                                    |
| Men's Tropical Plaid Short-Sleeve | UPF 50+              | Wrinkle-resistant, front and back cape venting, two front bellows pockets                                    |

Summary:
- Sun Shield Shirt: Offers UPF 50+ sun protection, wicks moisture, fits comfortably over swimsuit, and is abrasion resistant.
- Men's Plaid Tropic Shirt: Provides UPF 50+ sun protection, is wrinkle-free, evaporates perspiration, and has front and back cape venting with two front bellows pockets.
- Men's TropicVibe Shirt: Features UPF 50+ sun protection, is wrinkle resistant, has front and back cape venting, and includes two front bellows pockets.
- Men's Tropical Plaid Short-Sleeve Shirt: Offers UPF 50+ sun protection, is wrinkle-resistant, has front and back cape venting, and includes two front bellows pockets.

## Retrieval QA Chain

In [28]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [29]:
query =  "Please create a markdown table listing all the shirts that offer sun protection. \
For each shirt, include the name, sun protection rating, and any other relevant details. After the table, provide a brief \
summary of the features of each shirt."

In [30]:
response = qa_stuff.invoke(query)
response



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Please create a markdown table listing all the shirts that offer sun protection. For each shirt, include the name, sun protection rating, and any other relevant details. After the table, provide a brief summary of the features of each shirt.',
 'result': "| Name                                | Sun Protection Rating | Additional Details                                                                                                      |\n|-------------------------------------|-----------------------|------------------------------------------------------------------------------------------------------------------------|\n| Sun Shield Shirt by                 | UPF 50+              | Made of 78% nylon, 22% Lycra Xtra Life fiber. Wicks moisture, abrasion-resistant, fits comfortably over swimsuit.       |\n| Men's Tropical Plaid Short-Sleeve   | UPF 50+              | Made of 100% polyester, wrinkle-resistant. Front and back cape venting, two front bellows pockets.             

In [31]:
display(Markdown(response['result']))

| Name                                | Sun Protection Rating | Additional Details                                                                                                      |
|-------------------------------------|-----------------------|------------------------------------------------------------------------------------------------------------------------|
| Sun Shield Shirt by                 | UPF 50+              | Made of 78% nylon, 22% Lycra Xtra Life fiber. Wicks moisture, abrasion-resistant, fits comfortably over swimsuit.       |
| Men's Tropical Plaid Short-Sleeve   | UPF 50+              | Made of 100% polyester, wrinkle-resistant. Front and back cape venting, two front bellows pockets.                        |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+            | Made of 52% polyester, 48% nylon. Wrinkle-free, quick-drying. Front and back cape venting, two front bellows pockets.     |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+              | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh. Wrinkle-resistant, front and back cape venting.        |

**Sun Shield Shirt by:** This shirt offers UPF 50+ sun protection, made of nylon and Lycra Xtra Life fiber. It wicks moisture, is abrasion-resistant, and fits comfortably over swimsuits.

**Men's Tropical Plaid Short-Sleeve Shirt:** With a UPF 50+ rating, this shirt is made of polyester and is wrinkle-resistant. It features front and back cape venting and two front bellows pockets.

**Men's Plaid Tropic Shirt, Short-Sleeve:** Providing UPF 50+ coverage, this shirt is a blend of polyester and nylon. It is wrinkle-free, quick-drying, and has front and back cape venting with two front bellows pockets.

**Men's TropicVibe Shirt, Short-Sleeve:** This shirt has a UPF 50+ rating, made of nylon, polyester, and knit mesh lining. It is wrinkle-resistant, with front and back cape venting and two front bellows pockets.

In [32]:
response = index.query(query, llm=llm)
response

"| Name                                | Sun Protection Rating | Additional Details                                                                                           |\n|-------------------------------------|-----------------------|--------------------------------------------------------------------------------------------------------------|\n| Sun Shield Shirt by                 | UPF 50+              | Made of 78% nylon, 22% Lycra Xtra Life fiber. Wicks moisture, abrasion-resistant, fits comfortably over swimsuit. |\n| Men's Tropical Plaid Short-Sleeve   | UPF 50+              | Made of 100% polyester, wrinkle-resistant. Front and back cape venting, two front bellows pockets.              |\n| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+            | Made of 52% polyester, 48% nylon. Wrinkle-free, quick-drying, front and back cape venting, two front bellows pockets. |\n| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+              | Shell: 71% Nylon, 29% Polyester. Lining: